In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
client.database_names()
db = client.data_diggers
collection = db.training_data


## import the package and data

In [11]:
from pprint import pprint
import numpy
import pymongo
import pandas as pd
from sklearn import linear_model
import sklearn

filter_ = {
                            'house_price':{'$exists':'true'},
                              'total_jobs':{'$exists':'true'},
                              'school_number':{'$exists':'true'},
                              'unemployment':{'$exists':'true'},
                              'traffic_noise':{'$exists':'true'},
                              'hospitals_number':{'$exists':'true'},
                              'population':{'$exists':'true'},
                              'GVA':{'$exists':'true'},
                              'stations_number':{'$exists':'true'},
                              'happiness':{'$exists':'true'},
                              'pubs_number':{'$exists':'true'},
                              'road_traffic_2016':{'$exists':'true'},
#                               'best_Uni_ranking':{'$exists':'true'}
                             }

all_found = collection.find(filter_)


# read data to pandas
data = pd.DataFrame(list(all_found))
# print(data[0:10])

# create target variable
target = data.iloc[:, 4]
# print(target)


# remove happiness from data
data = data.drop(['happiness'], axis=1)

data = data.fillna(0)
# create target variable
# print(data[0:10])

# remove some columns
data = data.drop(['_id', 'city_name', 'latitude', 'longitude', 'region', 'sample', 'uk_region'], axis=1)

print(data[0:5])

data_pandas = data
# convert both to Numpy array representation
data = data.as_matrix()
target = target.as_matrix()

        GVA  best_Uni_ranking  hospitals_number  house_price  population  \
0  0.168831          0.797753          0.177215    -0.570112    1.567995   
1  0.168831          0.797753          0.177215    -0.570112    1.567995   
2  0.201299          0.000000          0.000000    -1.978730   -0.299514   
3  0.139610          0.258427          0.088608    -0.807119   -0.011484   
4  0.139610          0.258427          0.088608    -0.807119   -0.011484   

   pubs_number  road_traffic_2015  road_traffic_2016  school_number  \
0     0.644543           0.429722           0.441331       0.266980   
1     0.644543           0.429722           0.441331       0.266980   
2     0.151917          -0.499028          -0.485096       0.030303   
3     0.699115           0.032681           0.095372       0.086207   
4     0.699115           0.032681           0.095372       0.092476   

   stations_number  total_jobs  traffic_noise  unemployment  
0         0.578947    1.188895       1.084697      1.3

# Linear regression : Lasso

In [17]:
# run lasso for feature selection
clf = linear_model.Lasso(alpha=0.0005)
clf.fit(data, target)

alphas = numpy.geomspace(0.00001, 0.05, num=100)
print(alphas)

error = []
for i in alphas:
    
    mean_error = []
    
#     cross validation
    kf = sklearn.model_selection.KFold(4, shuffle=True)
    for train_index, test_index in kf.split(data):
        X_train, X_test = data[train_index], data[test_index]
        Y_train, Y_test = target[train_index], target[test_index]
        
#         train model
        clf = linear_model.Lasso(alpha=i)
        clf.fit(X_train, Y_train)
        
#         error
        mean_error.append(sklearn.metrics.mean_squared_error(Y_test, clf.predict(X_test)))
    
#     error
    error.append(numpy.mean(mean_error))

    
print(error)
print(error.index(min(error)))
print(alphas[error.index(min(error))])

# alpha
alpha = alphas[error.index(min(error))]

clf = linear_model.Lasso(alpha=alpha)
clf.fit(data, target)


weights = clf.coef_
print(weights.tolist())
print(clf.intercept_)

[  1.00000000e-05   1.08984148e-05   1.18775445e-05   1.29446407e-05
   1.41076064e-05   1.53750546e-05   1.67563723e-05   1.82617896e-05
   1.99024558e-05   2.16905218e-05   2.36392304e-05   2.57630139e-05
   2.80776012e-05   3.06001344e-05   3.33492958e-05   3.63454458e-05
   3.96107745e-05   4.31694651e-05   4.70478737e-05   5.12747243e-05
   5.58813215e-05   6.09017821e-05   6.63732883e-05   7.23363628e-05
   7.88351687e-05   8.59178369e-05   9.36368225e-05   1.02049293e-04
   1.11217553e-04   1.21209502e-04   1.32099143e-04   1.43967126e-04
   1.56901346e-04   1.70997595e-04   1.86360272e-04   2.03103154e-04
   2.21350242e-04   2.41236676e-04   2.62909736e-04   2.86529935e-04
   3.12272209e-04   3.40327206e-04   3.70902706e-04   4.04225154e-04
   4.40541340e-04   4.80120226e-04   5.23254938e-04   5.70264936e-04
   6.21498382e-04   6.77334716e-04   7.38187469e-04   8.04507324e-04
   8.76785453e-04   9.55557156e-04   1.04140582e-03   1.13496727e-03
   1.23693440e-03   1.34806242e-03

/usr/local/anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[0.026633766548644029, 0.021614262515292713, 0.027247247977517604, 0.032694457267319565, 0.029408288305953045, 0.028683350837394608, 0.020115138870228578, 0.023184405729493228, 0.018825302604551446, 0.024596865096693978, 0.026564519411770147, 0.047088986902007864, 0.018445530794854167, 0.020216632432110089, 0.024004834473283981, 0.037535453101578367, 0.023461189663711031, 0.024479584308554271, 0.019563319173930997, 0.023535025487414148, 0.031146658685405108, 0.025357524175662574, 0.019512281198538758, 0.021276198561273198, 0.041377169888328916, 0.019902761153585812, 0.046261292668400768, 0.02289785815693365, 0.026432820218834453, 0.022058728928588546, 0.022652240737563372, 0.027102490752222645, 0.019268475236673226, 0.032761925362331794, 0.029917711062796642, 0.027062838521674737, 0.029992916606799133, 0.024295309801233969, 0.028431221217149569, 0.028110325364665168, 0.021135860081202551, 0.026103889794553116, 0.020395040426256294, 0.023903169249973506, 0.026254292239093344, 0.02325378

In [18]:
# select the features
boundary = 0.000001

selectedFeatures = []
data_deleted = data_pandas

for i in range(len(weights)):
    if abs(weights[i])> boundary:
        selectedFeatures.append(data_pandas.columns.values[i])
    else:
        data_deleted = data_deleted.drop(data_pandas.columns.values[i],axis=1)

In [19]:
# convert both to Numpy array representation
data_d = data_deleted.as_matrix()
clf = linear_model.Lasso(alpha=alpha)
clf.fit(data_d, target)
weights = clf.coef_
print(weights.tolist())
print(clf.intercept_)
data_deleted.columns

[-0.4605009525701098, 0.019925847091217242, 0.004058816577515066, -0.15624585015318318, -0.11466159306996417, -0.5031451058300254, 0.09782069486987625, -0.09365879119118531]
0.640084435036


Index(['GVA', 'house_price', 'population', 'pubs_number', 'road_traffic_2015',
       'stations_number', 'traffic_noise', 'unemployment'],
      dtype='object')

In [21]:
from pprint import pprint

# save features to dictionary
values = weights.tolist()
keys = data_deleted.columns

count = 0
valueFeatures = {}
for key in keys:
    valueFeatures[key] = values[count]
    count += 1


sortedFeatures = sorted(valueFeatures, key=lambda dict_key: abs(valueFeatures[dict_key]))
print("The rank of importance: less to most")  
pprint(sortedFeatures)
valueFeatures

The rank of importance: less to most
['population',
 'house_price',
 'unemployment',
 'traffic_noise',
 'road_traffic_2015',
 'pubs_number',
 'GVA',
 'stations_number']


{'GVA': -0.4605009525701098,
 'house_price': 0.019925847091217242,
 'population': 0.004058816577515066,
 'pubs_number': -0.15624585015318318,
 'road_traffic_2015': -0.11466159306996417,
 'stations_number': -0.5031451058300254,
 'traffic_noise': 0.09782069486987625,
 'unemployment': -0.09365879119118531}

## Randomforest

In [37]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=3, random_state=0)
regr.fit(data, target)
importance_rank_randomForest = regr.feature_importances_.tolist()
keys = list(data_pandas.columns)
randomForest_result = {}
for i in range(len(importance_rank_randomForest)):
    randomForest_result[keys[i]] = importance_rank_randomForest[i]
sortedFeatures = sorted(randomForest_result, key=lambda dict_key: abs(randomForest_result[dict_key]))
print("The rank of importance: less to most")  
pprint(sortedFeatures)
randomForest_result

The rank of importance: less to most
['best_Uni_ranking',
 'pubs_number',
 'traffic_noise',
 'school_number',
 'road_traffic_2015',
 'GVA',
 'hospitals_number',
 'road_traffic_2016',
 'total_jobs',
 'stations_number',
 'unemployment',
 'house_price',
 'population']


{'GVA': 0.035970014063121405,
 'best_Uni_ranking': 0.0,
 'hospitals_number': 0.05654761469025025,
 'house_price': 0.14547038335887041,
 'population': 0.3122148174260362,
 'pubs_number': 0.004971677845466304,
 'road_traffic_2015': 0.025568818839859602,
 'road_traffic_2016': 0.06029490151770102,
 'school_number': 0.024789253006715316,
 'stations_number': 0.099344102421776,
 'total_jobs': 0.08376470666629357,
 'traffic_noise': 0.01298410719976074,
 'unemployment': 0.13807960296414917}

## chisquare

In [39]:
from scipy.stats import chisquare 
test_chis = chisquare(data)
print(test_chis)

Power_divergenceResult(statistic=array([  1.71251648e+01,   3.86439583e+01,   3.81900832e+00,
        -5.53781707e+01,  -1.23390422e+02,   1.53210289e+01,
        -5.82165680e+01,  -6.71814129e+01,   2.28691364e+00,
         8.96068223e+00,  -8.55873358e+01,   1.14185125e+03,
         4.97977329e+03]), pvalue=array([  1.00000000e+000,   9.99982537e-001,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   7.19221076e-187,
         0.00000000e+000]))


## pearsonr

In [44]:
from scipy.stats import pearsonr
keys = list(data_pandas.columns)
for i in range(len(data[0,:])):
    p = pearsonr(data[:,i],target)
    print("the %s pearsonr correlation coefficient is %f, p-value is %f " % (keys[i],p[0],p[1]))

the GVA pearsonr correlation coefficient is -0.098760, p-value is 0.377376 
the best_Uni_ranking pearsonr correlation coefficient is -0.171256, p-value is 0.123957 
the hospitals_number pearsonr correlation coefficient is -0.360762, p-value is 0.000870 
the house_price pearsonr correlation coefficient is 0.144106, p-value is 0.196477 
the population pearsonr correlation coefficient is -0.357872, p-value is 0.000964 
the pubs_number pearsonr correlation coefficient is -0.430021, p-value is 0.000055 
the road_traffic_2015 pearsonr correlation coefficient is -0.442338, p-value is 0.000032 
the road_traffic_2016 pearsonr correlation coefficient is -0.436806, p-value is 0.000041 
the school_number pearsonr correlation coefficient is -0.410237, p-value is 0.000129 
the stations_number pearsonr correlation coefficient is -0.517841, p-value is 0.000001 
the total_jobs pearsonr correlation coefficient is -0.351855, p-value is 0.001189 
the traffic_noise pearsonr correlation coefficient is -0.31

In [45]:
for i in range(len(data[0,:])):
    for j in range(len(data[0,:])):
        p = pearsonr(data[:,i],data[:,j])
        print("the %s and %s pearsonr correlation coefficient is %f, p-value is %f " % (keys[i],keys[j],p[0],p[1]))

the GVA and GVA pearsonr correlation coefficient is 1.000000, p-value is 0.000000 
the GVA and best_Uni_ranking pearsonr correlation coefficient is -0.249244, p-value is 0.023939 
the GVA and hospitals_number pearsonr correlation coefficient is -0.259872, p-value is 0.018385 
the GVA and house_price pearsonr correlation coefficient is 0.668092, p-value is 0.000000 
the GVA and population pearsonr correlation coefficient is -0.236652, p-value is 0.032308 
the GVA and pubs_number pearsonr correlation coefficient is -0.397154, p-value is 0.000220 
the GVA and road_traffic_2015 pearsonr correlation coefficient is -0.221969, p-value is 0.045045 
the GVA and road_traffic_2016 pearsonr correlation coefficient is -0.178238, p-value is 0.109135 
the GVA and school_number pearsonr correlation coefficient is -0.236017, p-value is 0.032788 
the GVA and stations_number pearsonr correlation coefficient is -0.344648, p-value is 0.001520 
the GVA and total_jobs pearsonr correlation coefficient is -0.1